## Setup

### Import TensorFlow and other libraries

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras

import numpy as np
import os
import time

### Load the dataset

In [2]:
path_to_file = tf.keras.utils.get_file('one_to_45.txt', 'https://raw.githubusercontent.com/nurseiit/46th-soz/master/one_to_45.txt')

### Read the data

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 117617 characters


In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

86 unique characters


In [5]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

## Generate text

In [6]:
# Reload the model from the 2 files we saved
with open('saved_model/model_config.json') as json_file:
    json_config = json_file.read()
model = keras.models.model_from_json(json_config)
model.load_weights('saved_model/model_weights.h5')

model.build(tf.TensorShape([1, None]))

In [7]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            22016     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (1, None, 86)             88150     
Total params: 4,048,470
Trainable params: 4,048,470
Non-trainable params: 0
_________________________________________________________________


In [8]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.7

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [9]:
result = generate_text(model, start_string=u"Астана ")
# Make sure the result ends correctly with a period
result = result[:result.rfind('.') + 1]
print(result)

Астана табу керек. Езді көңілің бір ғана инанмақтықпен қалған сөзге ұқсатамын. Тегінді осы екеуіне лайықты өнер, мал табу керек, онан соң ар-болса, кімнен көрек, көңілі танымақтаншықтың, қайру, Біреуі - білсем екен демеклік. Не көрсе соған талап қылады. Ондай нәрсе болады. Мұндай қиял өзі де қатпа бірлік есем емес пе? Осындай біз мәзі болады. Біреуі жоқ болса, да көп өзі ойлап жасағанына кіреді, аса арсыздана алмайсың.

Досы жоқпен сырлас,
Досы көппен сыйлас;

Әй, Ақиқтаң бар болса, қай алмайсың! Құдай тағала саған еңбек қылып мал табарлық қуат бар.

Оның қауын өзі білмей таласқандай қылатұғыны қалай?

Біреудің ағабын тұп, өз сауыбы асырай білесің бе? Жоқ, қазақ ортасында да ұрлық, өтірік, өсек, қастық пен биекельнамай деп пе? Ғалым шайратсыздық.



Зин адамға мұқтәди болсын.
